# Notebook 02: Visualización de Imágenes CT Pulmonares

Este notebook cubre técnicas de visualización para análisis de imágenes CT:

1. **Visualización de slices**: Exploración de volúmenes 3D
2. **Anotaciones de nódulos**: Marcado de regiones de interés
3. **Comparación NDCT vs LDCT**: Análisis de diferencias entre dosis
4. **Máscaras de segmentación**: Visualización de resultados
5. **Visualización 3D**: Rendering volumétrico

---

## Configuración del entorno

In [ ]:
# Detectar si estamos en Google Colab
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔵 Ejecutando en Google Colab")
    from google.colab import drive
    drive.mount('/content/drive')
    
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
else:
    print("🟢 Ejecutando localmente")
    import os
    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)

## Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

# Importar nuestros módulos
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer

print("✅ Librerías importadas correctamente")

## 1. Configuración de rutas

In [ ]:
import os

if IN_COLAB:
    # ===== CONFIGURACIÓN PARA GOOGLE COLAB =====
    DATA_PATH = '/content/drive/MyDrive/LUNA16/subset0'
    ANNOTATIONS_PATH = '/content/drive/MyDrive/LUNA16/annotations.csv'
    
else:
    # ===== CONFIGURACIÓN LOCAL =====
    # Estructura: Imagenes Biomedicas/notebooks/ y Imagenes Biomedicas/LUNA16/
    
    project_root = os.path.abspath('..')
    
    possible_paths = [
        os.path.join(project_root, 'LUNA16', 'subset0'),  # Recomendado
        os.path.join(project_root, 'data', 'LUNA16', 'subset0'),
        os.path.join(os.path.expanduser('~'), 'Desktop', 'LUNA16', 'subset0'),
    ]
    
    DATA_PATH = None
    for path in possible_paths:
        if os.path.exists(path):
            DATA_PATH = path
            print(f"✅ Datos encontrados: {DATA_PATH}")
            break
    
    if DATA_PATH is None:
        print("⚠️  No se encontró LUNA16. Recomendación:")
        print(f"   Coloca LUNA16 en: {os.path.join(project_root, 'LUNA16', 'subset0')}")
        DATA_PATH = input("\n📝 Ingresa la ruta: ").strip().strip('"').strip("'")
        if not os.path.exists(DATA_PATH):
            raise FileNotFoundError(f"❌ Ruta no existe: {DATA_PATH}")
    
    # Buscar anotaciones
    luna16_root = os.path.dirname(DATA_PATH)
    possible_annotation_paths = [
        os.path.join(luna16_root, 'annotations.csv'),
        os.path.join(project_root, 'LUNA16', 'annotations.csv'),
    ]
    
    ANNOTATIONS_PATH = None
    for path in possible_annotation_paths:
        if os.path.exists(path):
            ANNOTATIONS_PATH = path
            print(f"✅ Anotaciones: {ANNOTATIONS_PATH}")
            break
    
    if ANNOTATIONS_PATH is None:
        print("⚠️  No se encontró annotations.csv (opcional)")

# Detectar primer archivo .mhd
if os.path.exists(DATA_PATH):
    mhd_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.mhd')]
    if mhd_files:
        EXAMPLE_SCAN = mhd_files[0]
        print(f"\n📁 Configuración:")
        print(f"  - Datos: {DATA_PATH}")
        print(f"  - Anotaciones: {ANNOTATIONS_PATH if ANNOTATIONS_PATH else 'N/A'}")
        print(f"  - Archivos .mhd: {len(mhd_files)}")
        print(f"  - Ejemplo: {EXAMPLE_SCAN}")
    else:
        raise FileNotFoundError(f"❌ No hay archivos .mhd en {DATA_PATH}")
else:
    raise FileNotFoundError(f"❌ Directorio no existe: {DATA_PATH}")

## 2. Cargar datos y anotaciones

In [ ]:
# Inicializar cargador y preprocesador
loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)
preprocessor = LungPreprocessor()
visualizer = LungVisualizer()

# Cargar escaneo
import os
scan_path = os.path.join(DATA_PATH, EXAMPLE_SCAN)
ct_scan, origin, spacing = loader.load_itk_image(scan_path)

# Obtener seriesuid del nombre de archivo
seriesuid = EXAMPLE_SCAN.replace('.mhd', '')

# Cargar anotaciones para este escaneo
annotations = loader.get_annotations_for_scan(seriesuid)

print(f"\n📊 Volumen CT: {ct_scan.shape}")
if annotations is not None and len(annotations) > 0:
    print(f"📍 Nódulos anotados: {len(annotations)}")
    print(f"\nPrimeros nódulos:")
    print(annotations[['coordX', 'coordY', 'coordZ', 'diameter_mm']].head())
else:
    print("ℹ️  No hay anotaciones para este escaneo")

## 3. Visualización básica de slices

### 3.1 Slice único con detalles

In [ ]:
# Seleccionar slice del medio
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]

# Crear figura con información detallada
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Slice original
im1 = axes[0].imshow(ct_slice, cmap='bone')
axes[0].set_title(f'Slice {slice_idx}/{ct_scan.shape[0]}', fontsize=14)
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], label='HU', fraction=0.046)

# Histograma de valores HU
axes[1].hist(ct_slice.flatten(), bins=100, color='steelblue', alpha=0.7)
axes[1].axvline(-320, color='red', linestyle='--', linewidth=2, label='Threshold pulmonar (-320 HU)')
axes[1].axvline(0, color='blue', linestyle='--', linewidth=2, label='Agua (0 HU)')
axes[1].set_xlabel('Unidades Hounsfield (HU)', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribución de valores HU', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Estadísticas del slice:")
print(f"  - Min HU: {ct_slice.min():.1f}")
print(f"  - Max HU: {ct_slice.max():.1f}")
print(f"  - Media HU: {ct_slice.mean():.1f}")
print(f"  - Desv. Est.: {ct_slice.std():.1f}")

### 3.2 Múltiples slices del volumen

In [ ]:
# Usar la función del visualizador
visualizer.plot_volume_slices(
    ct_scan, 
    num_slices=9, 
    cmap='bone',
    title=f'Volumen CT - {seriesuid[:20]}...'
)

## 4. Visualización con anotaciones de nódulos

### 4.1 Convertir coordenadas mundo → voxel

In [ ]:
if annotations is not None and len(annotations) > 0:
    # Convertir todas las anotaciones a coordenadas voxel
    annotations_voxel = []
    
    for idx, row in annotations.iterrows():
        # Coordenadas mundo (mm)
        world_coords = np.array([row['coordZ'], row['coordY'], row['coordX']])
        
        # Convertir a voxel
        voxel_coords = loader.world_to_voxel(world_coords, origin, spacing)
        
        annotations_voxel.append({
            'z': voxel_coords[0],
            'y': voxel_coords[1],
            'x': voxel_coords[2],
            'diameter': row['diameter_mm']
        })
    
    print(f"✅ {len(annotations_voxel)} anotaciones convertidas a coordenadas voxel")
    print(f"\nEjemplo - Primer nódulo:")
    print(f"  - Mundo (mm): Z={annotations.iloc[0]['coordZ']:.2f}, Y={annotations.iloc[0]['coordY']:.2f}, X={annotations.iloc[0]['coordX']:.2f}")
    print(f"  - Voxel: Z={annotations_voxel[0]['z']}, Y={annotations_voxel[0]['y']}, X={annotations_voxel[0]['x']}")
    print(f"  - Diámetro: {annotations_voxel[0]['diameter']:.2f} mm")
else:
    print("ℹ️  No hay anotaciones para visualizar")
    annotations_voxel = []

### 4.2 Visualizar slice con nódulo anotado

In [ ]:
if len(annotations_voxel) > 0:
    # Seleccionar el primer nódulo
    nodule = annotations_voxel[0]
    nodule_slice_idx = nodule['z']
    
    # Cargar slice que contiene el nódulo
    ct_slice_nodule = ct_scan[nodule_slice_idx]
    
    # Segmentar pulmones
    lung_mask = preprocessor.segment_lung_mask(ct_slice_nodule)
    
    # Preparar anotaciones para este slice
    slice_annotations = [{
        'x': nodule['x'],
        'y': nodule['y'],
        'diameter': nodule['diameter'] / spacing[1]  # Convertir mm a píxeles
    }]
    
    # Usar función del visualizador
    visualizer.plot_ct_with_annotations(
        ct_slice_nodule,
        lung_mask=lung_mask,
        annotations=slice_annotations,
        title=f"Slice {nodule_slice_idx} - Nódulo de {nodule['diameter']:.1f}mm",
        figsize=(16, 5)
    )
else:
    print("ℹ️  No hay nódulos para visualizar")

### 4.3 Visualizar todos los nódulos del volumen

In [ ]:
if len(annotations_voxel) > 0:
    num_nodules = min(len(annotations_voxel), 6)  # Máximo 6 nódulos
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx in range(num_nodules):
        nodule = annotations_voxel[idx]
        z_idx = nodule['z']
        
        # Extraer región alrededor del nódulo (zoom)
        y_center, x_center = int(nodule['y']), int(nodule['x'])
        crop_size = 64
        
        y_start = max(0, y_center - crop_size // 2)
        y_end = min(ct_scan.shape[1], y_center + crop_size // 2)
        x_start = max(0, x_center - crop_size // 2)
        x_end = min(ct_scan.shape[2], x_center + crop_size // 2)
        
        ct_crop = ct_scan[z_idx, y_start:y_end, x_start:x_end]
        
        # Visualizar
        axes[idx].imshow(ct_crop, cmap='bone')
        
        # Dibujar círculo centrado
        radius_pixels = (nodule['diameter'] / 2) / spacing[1]
        circle = Circle(
            (crop_size // 2, crop_size // 2),
            radius_pixels,
            color='red',
            fill=False,
            linewidth=2
        )
        axes[idx].add_patch(circle)
        
        axes[idx].set_title(f'Nódulo {idx+1}\nSlice {z_idx}, Ø={nodule["diameter"]:.1f}mm', fontsize=11)
        axes[idx].axis('off')
    
    # Ocultar ejes vacíos
    for idx in range(num_nodules, 6):
        axes[idx].axis('off')
    
    plt.suptitle(f'Nódulos Anotados - {seriesuid[:30]}...', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("ℹ️  No hay nódulos para visualizar")

## 5. Comparación NDCT vs LDCT

Si tienes pares de imágenes NDCT (Normal Dose) y LDCT (Low Dose), puedes compararlas:

In [ ]:
# Ejemplo: Simulamos LDCT añadiendo ruido al NDCT
# (En un caso real, cargarías el archivo LDCT correspondiente)

ndct_slice = ct_scan[ct_scan.shape[0] // 2]

# Simular LDCT con ruido gaussiano
noise_level = 50  # HU
noise = np.random.normal(0, noise_level, ndct_slice.shape)
ldct_slice = ndct_slice + noise

# Visualizar comparación
visualizer.compare_ndct_ldct(
    ndct_slice,
    ldct_slice,
    title="Comparación NDCT vs LDCT (Simulado)",
    figsize=(15, 5)
)

# Calcular métricas de diferencia
mse = np.mean((ndct_slice - ldct_slice) ** 2)
psnr = 10 * np.log10(np.max(ndct_slice) ** 2 / mse)

print(f"\n📊 Métricas de diferencia:")
print(f"  - MSE (Mean Squared Error): {mse:.2f}")
print(f"  - PSNR (Peak Signal-to-Noise Ratio): {psnr:.2f} dB")
print(f"  - Diferencia media: {np.mean(np.abs(ndct_slice - ldct_slice)):.2f} HU")

## 6. Visualización de máscaras de segmentación

### 6.1 Segmentación de múltiples slices

In [ ]:
# Seleccionar 9 slices distribuidos uniformemente
num_slices = 9
step = ct_scan.shape[0] // num_slices
slice_indices = range(0, ct_scan.shape[0], step)[:num_slices]

# Crear visualización
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for idx, slice_idx in enumerate(slice_indices):
    ct_slice = ct_scan[slice_idx]
    lung_mask = preprocessor.segment_lung_mask(ct_slice)
    
    # Visualizar superposición
    axes[idx].imshow(ct_slice, cmap='bone')
    axes[idx].imshow(lung_mask, cmap='Reds', alpha=0.3)
    
    # Calcular área pulmonar
    area_pixels = np.sum(lung_mask)
    area_mm2 = area_pixels * spacing[1] * spacing[2]
    
    axes[idx].set_title(f'Slice {slice_idx}\nÁrea: {area_mm2/100:.0f} cm²', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('Segmentación Pulmonar - Múltiples Slices', fontsize=16)
plt.tight_layout()
plt.show()

### 6.2 Evolución del área pulmonar a lo largo del volumen

In [ ]:
# Calcular área pulmonar en cada slice
lung_areas = []

print("Calculando áreas pulmonares...")
for i in range(0, ct_scan.shape[0], 5):  # Cada 5 slices para velocidad
    lung_mask = preprocessor.segment_lung_mask(ct_scan[i])
    area_mm2 = np.sum(lung_mask) * spacing[1] * spacing[2]
    lung_areas.append(area_mm2 / 100)  # Convertir a cm²

# Graficar
fig, ax = plt.subplots(figsize=(14, 5))

slice_positions = list(range(0, ct_scan.shape[0], 5))
ax.plot(slice_positions, lung_areas, linewidth=2, color='steelblue')
ax.fill_between(slice_positions, lung_areas, alpha=0.3, color='steelblue')

ax.set_xlabel('Número de Slice', fontsize=12)
ax.set_ylabel('Área Pulmonar (cm²)', fontsize=12)
ax.set_title('Evolución del Área Pulmonar a lo Largo del Volumen CT', fontsize=14)
ax.grid(True, alpha=0.3)

# Marcar máximo
max_area_idx = np.argmax(lung_areas)
max_slice = slice_positions[max_area_idx]
max_area = lung_areas[max_area_idx]
ax.axvline(max_slice, color='red', linestyle='--', alpha=0.7, label=f'Máx área: slice {max_slice}')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Estadísticas de área pulmonar:")
print(f"  - Área máxima: {max_area:.1f} cm² (slice {max_slice})")
print(f"  - Área media: {np.mean(lung_areas):.1f} cm²")
print(f"  - Área total estimada: {np.sum(lung_areas) * spacing[0] / 10:.1f} cm³")

## 7. Visualización 3D de segmentación pulmonar

Rendering 3D de la superficie pulmonar usando marching cubes:

In [ ]:
# Crear volumen 3D de máscaras pulmonares (muestreando para velocidad)
print("Generando volumen 3D de segmentación...")
lung_volume = np.zeros(ct_scan.shape, dtype=np.uint8)

for i in range(ct_scan.shape[0]):
    if i % 10 == 0:
        print(f"  Procesando slice {i}/{ct_scan.shape[0]}")
    lung_volume[i] = preprocessor.segment_lung_mask(ct_scan[i])

print("✅ Volumen 3D generado")

# Extraer superficie usando marching cubes
print("Extrayendo superficie 3D...")
verts, faces, normals, values = measure.marching_cubes(lung_volume, level=0.5, spacing=spacing)

# Crear visualización 3D
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

# Crear malla 3D
mesh = Poly3DCollection(verts[faces], alpha=0.3, edgecolor='none')
mesh.set_facecolor('cyan')
ax.add_collection3d(mesh)

# Configurar límites de los ejes
ax.set_xlim(0, ct_scan.shape[2] * spacing[2])
ax.set_ylim(0, ct_scan.shape[1] * spacing[1])
ax.set_zlim(0, ct_scan.shape[0] * spacing[0])

ax.set_xlabel('X (mm)', fontsize=10)
ax.set_ylabel('Y (mm)', fontsize=10)
ax.set_zlabel('Z (mm)', fontsize=10)
ax.set_title('Superficie 3D de Pulmones Segmentados', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\n📊 Información de la superficie 3D:")
print(f"  - Vértices: {len(verts)}")
print(f"  - Caras (triángulos): {len(faces)}")
print(f"  - Volumen aproximado: {np.sum(lung_volume) * np.prod(spacing) / 1000:.1f} cm³")

## 8. Visualización interactiva de slices (slider)

Explorar el volumen slice por slice con controles interactivos:

In [ ]:
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

def visualize_slice(slice_idx, show_mask, show_annotations):
    """
    Función para visualización interactiva de slices
    """
    ct_slice = ct_scan[slice_idx]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # CT original
    axes[0].imshow(ct_slice, cmap='bone')
    axes[0].set_title(f'Slice {slice_idx}/{ct_scan.shape[0]}', fontsize=14)
    axes[0].axis('off')
    
    # Con procesamiento
    axes[1].imshow(ct_slice, cmap='bone')
    
    # Mostrar máscara si está activado
    if show_mask:
        lung_mask = preprocessor.segment_lung_mask(ct_slice)
        axes[1].imshow(lung_mask, cmap='Reds', alpha=0.3)
    
    # Mostrar anotaciones si están activadas y existen
    if show_annotations and len(annotations_voxel) > 0:
        for nodule in annotations_voxel:
            if nodule['z'] == slice_idx:
                radius_pixels = (nodule['diameter'] / 2) / spacing[1]
                circle = Circle(
                    (nodule['x'], nodule['y']),
                    radius_pixels,
                    color='yellow',
                    fill=False,
                    linewidth=2
                )
                axes[1].add_patch(circle)
                axes[1].text(
                    nodule['x'], nodule['y'] - radius_pixels - 10,
                    f"{nodule['diameter']:.1f}mm",
                    color='yellow',
                    fontsize=10,
                    weight='bold'
                )
    
    title = 'Procesado'
    if show_mask:
        title += ' + Máscara'
    if show_annotations:
        title += ' + Anotaciones'
    axes[1].set_title(title, fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Crear controles interactivos
interact(
    visualize_slice,
    slice_idx=IntSlider(min=0, max=ct_scan.shape[0]-1, step=1, value=ct_scan.shape[0]//2, description='Slice:'),
    show_mask=widgets.Checkbox(value=True, description='Mostrar máscara pulmonar'),
    show_annotations=widgets.Checkbox(value=True, description='Mostrar nódulos')
);

## 9. Mapa de calor de densidad pulmonar

In [ ]:
# Seleccionar slice
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]
lung_mask = preprocessor.segment_lung_mask(ct_slice)

# Crear mapa de densidad (solo región pulmonar)
lung_density = ct_slice.copy()
lung_density[lung_mask == 0] = -1000  # Fondo a valor de aire

# Visualizar con múltiples colormaps
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

cmaps = ['bone', 'jet', 'viridis', 'hot']
titles = ['Bone (estándar CT)', 'Jet (densidad)', 'Viridis (perceptual)', 'Hot (calor)']

for idx, (cmap, title) in enumerate(zip(cmaps, titles)):
    ax = axes[idx // 2, idx % 2]
    im = ax.imshow(lung_density, cmap=cmap, vmin=-1000, vmax=200)
    ax.set_title(title, fontsize=12)
    ax.axis('off')
    plt.colorbar(im, ax=ax, label='HU', fraction=0.046)

plt.suptitle(f'Mapas de Densidad Pulmonar - Slice {slice_idx}', fontsize=16)
plt.tight_layout()
plt.show()

---

## Resumen

En este notebook has aprendido a:

1. ✅ Visualizar slices individuales y múltiples del volumen CT
2. ✅ Cargar y mostrar anotaciones de nódulos
3. ✅ Convertir entre coordenadas mundo y voxel
4. ✅ Comparar imágenes NDCT vs LDCT
5. ✅ Visualizar máscaras de segmentación
6. ✅ Analizar evolución del área pulmonar
7. ✅ Crear visualizaciones 3D de superficies
8. ✅ Usar controles interactivos para explorar volúmenes
9. ✅ Generar mapas de calor de densidad

**Próximos pasos**: Ver notebooks de segmentación y clasificación de nódulos.